# Vector Retriever

You will create a vector retriever using the Neo4j GraphRAG Python package with Microsoft Foundry.

You will be able to review how the vector index is used to retrieve similar results and how the context can be used by an LLM to provide a response.

---

Import the required Python modules and set up the Microsoft Foundry configuration.

In [ ]:
import sys
sys.path.insert(0, '../new-workshops/solutions')

from neo4j import GraphDatabase
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation import GraphRAG

from config import get_neo4j_driver, get_llm, get_embedder, Neo4jConfig

Create and verify the connection to your Neo4j graph database.

In [ ]:
neo4j_config = Neo4jConfig()
driver = GraphDatabase.driver(
    neo4j_config.uri, 
    auth=(
        neo4j_config.username, 
        neo4j_config.password
    ))
driver.verify_connectivity()

## Initialize LLM and Embedder

Set up the Large Language Model (LLM) and the embedding model you will use in retrieval-augmented generation (RAG) workflows.

- **LLM**: Uses Microsoft Foundry's model via the `OpenAILLM` interface with Azure CLI credentials.
- **Embedder**: Uses Microsoft Foundry's embedding API via the `OpenAIEmbeddings` class.

In [ ]:
# --- Initialize LLM and Embedder from Microsoft Foundry ---
llm = get_llm()
embedder = get_embedder()

## Initialize Vector Retriever

Set up the vector-based retriever for semantic search over your Neo4j knowledge graph.

> Vector search enables semantic retrieval of text chunks from your Neo4j graph.  
> Instead of keyword matching, it finds the most contextually similar passages to your query, even if the wording is different.

In [ ]:
# --- Initialize Vector Retriever ---
vector_retriever = VectorRetriever(
    driver=driver,
    index_name='chunkEmbeddings',
    embedder=embedder,
    return_properties=['text'])

The **VectorRetriever** class:
- Connects to the Neo4j database using the provided `driver`.
- Uses the `chunkEmbeddings` vector index for efficient semantic retrieval.
- The `embedder` generates embeddings for the query.
- Returns the `text` property from matching chunks.

> **Tip:**  
> You can modify the `return_properties` list to include additional properties from the retrieved nodes.

# Simple Vector Search Diagnostic 

You can use the vector retriever to search for semantically similar data.

Test the vector search by retrieving the top 10 most relevant text chunks from the Neo4j knowledge graph for the given query.

In [ ]:
# --- Simple Vector Search ---
query = "What are the risks that Apple faces?"
result = vector_retriever.search(query_text=query, top_k=10)
for item in result.items:
    print(f"Score: {item.metadata['score']:.4f}, Content: {item.content[0:100]}..., id: {item.metadata['id']}")

**How it works:**  
1. The example `query`, "What are the risks that Apple faces?", is created
2. `vector_retriever.search()` runs the query and returns the top 10 matches based on vector similarity.
3. The results are formatted displaying:
    * The similarity score (`Score`)
    * A snippet of the retrieved content (`Content`)
    * The unique identifier for each chunk (`id`)

This diagnostic helps you verify that the vector search is working and inspect the quality of the top results for your query.

> **Tip:**
> Inspecting the returned results to verify relevance can help you to adjust your chunking or embedding strategy.

## Retrieval-Augmented Generation (RAG) Query

You can use the `GraphRAG` class to create a retrieval-augmented generation (RAG) pipeline.

The `GraphRAG` class combines a Large Language Model (LLM) with a vector-based retriever to answer questions using both semantic search and generative reasoning.

In [ ]:
# --- Initialize RAG and Perform Search ---
query = "What are the risks that Apple faces?"
rag = GraphRAG(
    llm=llm,
    retriever=vector_retriever
)
response = rag.search(query)

print(response.answer)

- **How it works:**  
  1. The retriever (`vector_retriever`) finds the most relevant text chunks from the Neo4j graph based on the input query.
  2. The LLM (`llm`) uses the retrieved context to generate a natural language answer.
  3. The `rag` pipeline is used to `search`.
  4. The `answer` in the `response` is printed

The `GraphRAG` pipeline provides context-aware, accurate answers grounded in your knowledge graph data.

---

Experiment with the vector retriever by modifying the `query`, for example:
- What products does Microsoft reference?
- What warnings have Nvidia given?
- What companies mention AI in their filings?

[View the complete code](../new-workshops/solutions/02_01_vector_retriever.py)

[Move on to the Advanced RAG:Vector Cypher Retriever Notebook](02_02_vector_cypher_retriever.ipynb)

In [ ]:
# Cleanup
driver.close()